<a href="https://colab.research.google.com/github/zack-murray/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]



---

### Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

---



In [70]:
df['created'].head()

0    2016-06-24 07:54:24
1    2016-06-12 12:19:27
2    2016-04-17 03:26:41
3    2016-04-18 02:22:02
4    2016-04-28 01:32:41
Name: created, dtype: object

In [0]:
# Instantiate 'created' df converted to datetime
df['created'] = pd.to_datetime(df['created'])

In [0]:
# Convert strings into datetimes 
from datetime import datetime

april = datetime.strptime('2016-04-01', '%Y-%m-%d')
may = datetime.strptime('2016-05-31', '%Y-%m-%d') # set May to last day of month for condition in next cell
june = datetime.strptime('2016-06-01', '%Y-%m-%d')

In [73]:
# Engineer features for linear regression
df['bedbath'] = df['bedrooms'] + df['bathrooms']
df['animals'] = df['cats_allowed'] + df['dogs_allowed']
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bedbath,animals
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,0


In [0]:
# Split data into train and test datasets
train = df[(df['created'] >= april) & (df['created'] <= may)] # data on or after April 1st and on or before May 31st
test = df[df['created'] >= june]  #data pulled from the month of June



---

###Engineer at least two new features.

---



In [0]:
# Designate features for linear regression
features = ['bedbath', 'animals']
# Instantiate train and test models for features
X_train = train[features]
X_test = test[features]
# Designate target for linear regression
target = ['price']
# Instantiate train and test models for target
y_train = train[target]
y_test = test[target]



---

###Fit a linear regression model with at least two features.

---



In [0]:
from sklearn.linear_model import LinearRegression
# Instantiate model class
model = LinearRegression()
# Fit the model
model.fit(X_train, y_train)
# Apply the model to new data
y_pred1 = model.predict(X_train)
y_pred2 = model.predict(X_test)



---

###Get the model's coefficients and intercept.

---



In [80]:
print(f'The Intercept of the model is {model.intercept_}')
print(f'The coefficients of the model, for bedbath and animals, is {model.coef_} respectively.')

The Intercept of the model is, [1270.56693745]
The coefficients of the model, for bedbath and animals, is [[809.64095011 101.00456183]] respectively.




---

###Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.


---



In [89]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#RMSE, MAE, and R2 for training data
mse = mean_squared_error(train[target], y_pred1)
rmse = np.sqrt(mse)
mae = mean_absolute_error(train[target], y_pred1)
r2 = r2_score(train[target], y_pred1)
print(f'The RMSE of the training data is {rmse}')
print(f'The MAE of the training data is {mae}')
print(f'The R2 of the training data is {r2}')

The RMSE of the training data is 1337.218226474206
The MAE of the training data is 888.2612052297808
The R2 of the training data is 0.42417373055488405


In [90]:
#RMSE, MAE, and R2 for test data
mse = mean_squared_error(test[target], y_pred2)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test[target], y_pred2)
r2 = r2_score(test[target], y_pred2)
print(f'The RMSE of the test data is {rmse}')
print(f'The MAE of the test data is {mae}')
print(f'The R2 of the test data is {r2}')

The RMSE of the test data is 1336.221206601886
The MAE of the test data is 905.322683531065
The R2 of the test data is 0.4255228780315734
